In [1]:
import torch
import ChatTTS
from IPython.display import Audio

/root/miniconda3/envs/ChatTTS/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
chat = ChatTTS.Chat()
chat.load_models()

Fetching 11 files: 100%|██████████| 11/11 [00:00<00:00, 50313.35it/s]
INFO:ChatTTS.core:use cuda:0
INFO:ChatTTS.core:vocos loaded.
INFO:ChatTTS.core:dvae loaded.
INFO:ChatTTS.core:gpt loaded.
INFO:ChatTTS.core:decoder loaded.
INFO:ChatTTS.core:tokenizer loaded.
INFO:ChatTTS.core:All initialized.


In [3]:
texts = ["free memory equals to props's total memory minus torch dot cuda dot memory reserved",]*3 \
        + ["中国人民银行，国家金融监督管理总局发布通知。通知提出，对于贷款购买商品住房的居民家庭，首套住房商业性个人住房贷款最低首付款比例调整为不低于百分之十五",]*3

In [4]:
wavs = chat.infer(texts, use_decoder=True)

INFO:ChatTTS.core:All initialized.
 38%|███▊      | 773/2048 [00:16<00:27, 46.46it/s]


In [5]:
Audio(wavs[0], rate=24_000, autoplay=True)

In [25]:
from ChatTTS.experimental.llm import llm_api

API_KEY = 'sk-571b0e0042d64a329adb0925c82fa28d'
client = llm_api(api_key=API_KEY,
        base_url="https://api.deepseek.com",
        model="deepseek-chat")

In [26]:
user_question = '四川有哪些好吃的美食呢?'
text = client.call(user_question, prompt_version = 'deepseek')
text = client.call(text, prompt_version = 'deepseek_TN')

INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"
INFO:httpx:HTTP Request: POST https://api.deepseek.com/chat/completions "HTTP/1.1 200 OK"


In [139]:
spk_stat = torch.load('asset/spk_stat.pt')
torch.manual_seed(100)
rand_spk = torch.randn(768) * spk_stat.chunk(2)[0] + spk_stat.chunk(2)[1]

In [117]:
params_infer_code = {'spk_emb' : rand_spk, 'temperature':.3}
params_refine_text = {'prompt':'[oral_2][laugh_0][break_6]'}
# wav = chat.infer('四川美食可多了，有麻辣火锅、宫保鸡丁、麻婆豆腐、担担面、回锅肉、夫妻肺片等，每样都让人垂涎三尺。', params_refine_text=params_refine_text, params_infer_code=params_infer_code)
# wav = chat.infer('四川美食确实以辣闻名，但也有不辣的选择。比如甜水面、赖汤圆、蛋烘糕、叶儿粑等，这些小吃口味温和，甜而不腻，也很受欢迎。', params_refine_text=params_refine_text, params_infer_code=params_infer_code)
# wav = chat.infer("""
# chat T T S is a text to speech model designed for dialogue applications. 
# [uv_break]it supports mixed language input [uv_break]and offers multi speaker 
# capabilities with precise control over prosodic elements [laugh]like like 
# [uv_break]laughter[laugh], [uv_break]pauses, [uv_break]and intonation. 
# [uv_break]it delivers natural and expressive speech,[uv_break]so please
# [uv_break] use the project responsibly at your own risk.[uv_break]
# """.replace('\n', ''))
wav = chat.infer("What is [uv_break]your favorite english food?[laugh][lbreak]", params_refine_text=params_refine_text, params_infer_code=params_infer_code)
# wav = chat.infer("""大家好，我叫李明，非常高兴能在这里介绍自己。[uv_break]我来自美丽的天府之国成都，今年22岁，刚刚从四川大学毕业。
#                     [uv_break]在大学期间，我主修的是计算机科学。[uv_break]我一直对编程有着浓厚的兴趣，尤其是人工智能和数据分析方向。[uv_break]除了学习，我还积极参加各种课外活动，比如校内的编程大赛和志愿者服务。[laugh]
#                     [uv_break]我的性格比较开朗，喜欢结交新朋友。[uv_break] 我认为团队合作非常重要，因为它不仅能提高工作效率，还能让我们学到更多的知识和技能。[uv_break]在上一次的编程比赛中，我们小组就凭借出色的团队合作，拿到了全校的第一名。[laugh]
#                     [uv_break]在生活中，我有很多兴趣爱好，比如打篮球、听音乐和旅游。[uv_break]每个周末，我都会和朋友一起打篮球，这不仅锻炼了身体，也增进了我们的友谊。[laugh]
#                     [uv_break]总的来说，我是一个充满活力和热情的人，希望在未来的工作中能够发挥自己的特长，和大家一起成长。[uv_break]非常感谢大家的聆听！[laugh]""".replace('\n', ''))
# wav = chat.infer("""大家好，我是李明，22岁，南京大学计算机专业毕业。[uv_break]爱好programming、basketball、music 和travel。[laugh]期待与大家一起成长！[uv_break]谢谢！""")

INFO:ChatTTS.core:All initialized.
 15%|█▌        | 316/2048 [00:06<00:34, 50.11it/s]


In [118]:
Audio(wav[0], rate=24_000, autoplay=True)

In [40]:
print(wav[0][0])

[ 2.5386931e-04 -7.3818861e-05 -1.8749551e-04 ... -1.7170299e-07
 -1.2131311e-07 -1.5465558e-07]


In [43]:
import soundfile as sf
sf.write("data/test.wav", wav[0][0], 24_000, format="wav")